In [2]:
# importations
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter
from copy import deepcopy
import pickle
import timeit

## Paramètres de la simulation

In [3]:
L = 50 # Nombre de points sur chaque côté de la grille
n_pred0 = 10 # Nombre initial de prédateurs
n_proie0 = 200 # Nombre initial de proies
reserves0 = 6.0 # Réserves initiales données aux prédateurs initiaux
perte_par_cycle = 0.5 # Réserves perdues par le prédateur à chaque cycle
n_iter = 1000 # Nombre d'itérations

## Classes

In [4]:
class proie:
    '''
    Classe représentant une proie.
    '''
    VIE = 30
    AGE_DIVISION = 5
    
    def __init__(self, x, y, age):
        '''
        Constructeur de la classe proie.
        Initialise la position et l'âge de l'individu.
        '''
        self.x = x
        self.y = y
        self.age = age
    
    def __repr__(self):
        '''
        Permet d'afficher la position et l'âge de l'in-
        dividu sous forme de string.
        '''
        return f'[x:{self.x}, y:{self.y}, age:{self.age}]'
    
    def est_mort(self):
        '''
        Vérifie si la proie est morte se basant
        sur son age.
        
        :retour: État de la mort. (bool)
        '''
        return self.age > self.VIE
    
    def deplacements_possibles(self):
        '''
        Vérifie les déplacements possibles de l'individu
        en fonction de sa position afin de ne pas sortir
        du domaine.
        
        :retour: Liste de déplacements possibles classés dans
                 des tuples. (tuple in list)
        '''
        possibles = [
            (1,0), (1,1), (0,1), (-1,1), 
            (-1,0), (-1,-1), (0,-1), (1,-1)
        ]
        impossibles = []
        impossibles_x = []
        impossibles_y = []
        
        if self.x == 0:
            impossibles_x = [3,4,5]
        if self.x == L-1:
            impossibles_x = [0,1,7]
        if self.y == 0:
            impossibles_y = [5,6,7]
        if self.y == L-1:
            impossibles_y = [1,2,3]
            
        impossibles = np.unique(np.concatenate((impossibles_x, impossibles_y))).astype('int')
        possibles = np.delete(possibles, impossibles, axis=0)
        
        return possibles
    
    def vieillir(self):
        '''
        Fait vieillir la proie. Incrémente son age de 1.
        '''
        self.age += 1

In [5]:
class pred:
    '''
    Classe représentant un prédateur.
    '''
    VIE = 100
    AGE_DIVISION = 20
    RESERVES_DIVISION = 10
    RESERVES_SATIETE = 20
    
    def __init__(self, x, y, age, reserves):
        '''
        Constructeur de la classe pred.
        Initialise la position, l'âge et les réserves de l'individu.
        '''
        self.x = x
        self.y = y
        self.age = age
        self.reserves = reserves
        
    def __repr__(self):
        '''
        Permet d'afficher la position, l'âge et les
        réserves de l'individu sous forme de string.
        '''
        return f'[x:{self.x}, y:{self.y}, age:{self.age}, reserves:{self.reserves}]'
    
    def est_mort(self):
        '''
        Vérifie si la prédateur est mort se
        basant sur son age et ses réserves.
        
        :retour: État de la mort. (bool)
        '''
        return self.age > self.VIE or self.reserves <= 0

    def deplacements_possibles(self):
        '''
        Vérifie les déplacements possibles de l'individu
        en fonction de sa position afin de ne pas sortir
        du domaine.
        
        :retour: Liste de déplacements possibles classés dans
                 des tuples. (tuple in list)
        '''
        possibles = [
            (1,0), (1,1), (0,1), (-1,1), 
            (-1,0), (-1,-1), (0,-1), (1,-1)
        ]
        impossibles = []
        impossibles_x = []
        impossibles_y = []
        
        if self.x == 0:
            impossibles_x = [3,4,5]
        if self.x == L-1:
            impossibles_x = [0,1,7]
        if self.y == 0:
            impossibles_y = [5,6,7]
        if self.y == L-1:
            impossibles_y = [1,2,3]
            
        impossibles = np.unique(np.concatenate((impossibles_x, impossibles_y))).astype('int')
        possibles = np.delete(possibles, impossibles, axis=0)
        
        return possibles
    
    def vieillir(self):
        '''
        Fait vieillir le prédateur. Incrémente son age de 1
        et diminue ses réserves de la valeur de perte_par_cycle.
        '''
        self.age += 1
        self.reserves -= perte_par_cycle

In [6]:
class dict_proies:
    '''
    Classe qui gère la manipulation de l'ensemble des proies.
    '''    
        
    def __init__(self, nombre_initial):
        '''
        Constructeur de la classe dict_proies.
        
        Crée un dictionnaire de proies avec des 
        positions et des âges tirés aléatoirement.
        '''
        self.d_proies = {}
        
        for i in range(nombre_initial):
            x_ini = np.random.randint(0, L)
            y_ini = np.random.randint(0, L)
            age_ini = np.random.randint(0, proie.VIE+1)
            
            self.d_proies[(x_ini, y_ini)] = proie(x_ini, y_ini, age_ini)
            
    def __repr__(self):
        '''
        Permet de représenter la classe dict_proies sous
        la forme du dictionnaire qu'elle contient.
        '''
        return f'{self.d_proies}'
    
    def __len__(self):
        '''
        Définit la longueur de la classe dict_proies comme 
        étant la longueur du dictionnaire qu'elle contient.
        '''
        return len(self.d_proies)
    
    
###~~~FONCTIONS DU DÉPLACEMENT~~~###
    
   
    def verifier_position(self, coords, deplacement):
        '''
        Vérifie si la position post-déplacement est libre.
        
        :param coords: Les coordonnées actuelles de la proie.
        :param deplacement: Le déplacement à effectuer à partir
                            de la position courante.
                            
        :retour: Retourne si la position est libre. (bool)
        '''
        position = (
            coords[0] + deplacement[0],
            coords[1] + deplacement[1]
            )
       
        return position not in self.d_proies 

    
    def chercher_positions_vides(self, key):
        '''
        Cherche des positions vides aléatoirement à 5 reprises avant
        d'abandonner.
        
        :param key: La clé de l'objet pour lequel chercher des déplacements.
        
        :retour: Les coordonnées actuelles et les nouvelles sous forme de tuple
                 comme (vieilles, nouvelles).
        '''
        _deplacements_possibles = self.d_proies[key].deplacements_possibles()
        coordonnees = ()
        
        for i in range(5):
            deplacement_aleatoire = _deplacements_possibles[np.random.randint(0, len(_deplacements_possibles))]

            if self.verifier_position(key, deplacement_aleatoire):
                nouvelles_coords = (
                    key[0] + deplacement_aleatoire[0],
                    key[1] + deplacement_aleatoire[1]
                )

                coordonnees = (key, nouvelles_coords)
                break 
                    
        return coordonnees
        
        
###~~~FONCTIONS DE LA SIMULATION~~~###
       
    
    def eliminer_morts(self):
        '''
        Passer en revue les proies et éliminer celles qui
        qualifient comme étant mortes.
        '''
        a_enlever = []
        
        # trouver les proies mortes
        for key in self.d_proies.keys():
            if self.d_proies[key].est_mort():
                a_enlever.append(key)
                
        # éliminer les proies mortes
        for key in a_enlever:
            del self.d_proies[key]
    
    def diviser(self):
        '''
        Effectuer la division de chaque individu si son age et l'espace le permet.
        '''
        liste_keys = deepcopy(list(self.d_proies.keys()))
        
        for key in self.d_proies:
            if self.d_proies[key].age < proie.AGE_DIVISION:
                liste_keys.remove(key)
        
        for key in liste_keys:
            coords = self.chercher_positions_vides(key)
            
            if coords != ():
                self.d_proies[coords[0]] = proie(coords[0][0], coords[0][1], 0)
                self.d_proies[coords[1]] = proie(coords[1][0], coords[1][1], 0)
    
    
    def incrementer_temps(self):
        '''
        Appliquer le vieillissement à chaque proie.
        '''
        for individu_proie in self.d_proies.values():
            individu_proie.vieillir()

In [7]:
class dict_preds:
    '''
    Classe qui gère la manipulation de l'ensemble des prédateurs.
    '''
    
    def __init__(self, nombre_initial):
        '''
        Constructeur de la classe dict_preds.
        
        Crée un dictionnaire de prédateurs avec des 
        positions et des âges tirés aléatoirement avec 
        des réserves initiales identiques (reserves_0).
        '''
        self.d_preds = {}
        
        for i in range(nombre_initial):
            x_ini = np.random.randint(0, L)
            y_ini = np.random.randint(0, L)
            age_ini = np.random.randint(0, pred.VIE+1)
            
            self.d_preds[(x_ini, y_ini)] = pred(x_ini, y_ini, age_ini, reserves0)
            
    def __repr__(self):
        '''
        Permet de représenter la classe dict_preds sous
        la forme du dictionnaire qu'elle contient.
        '''
        return f'{self.d_preds}'
    
    def __len__(self):
        '''
        Définit la longueur de la classe dict_preds comme 
        étant la longueur du dictionnaire qu'elle contient.
        '''
        return len(self.d_preds)
    
    
###~~~FONCTIONS DU DÉPLACEMENT~~~###
    
    
    def verifier_position(self, coords, deplacement):
        '''
        Vérifie si la position post-déplacement est libre.
        
        :param coords: Les coordonnées actuelles du prédateur.
        :param deplacement: Le déplacement à effectuer à partir
                            de la position courante.
                            
        :retour: Retourne si la position est libre. (bool)
        '''
        position = (
            coords[0] + deplacement[0],
            coords[1] + deplacement[1]
            )
       
        return position not in self.d_preds 
    
    def changer_position(self, vieilles_coords, nouvelles_coords):
        '''
        Mets à jour la position d'un prédateur.
        
        :param vieilles_coords: Les coordonnées actuelles du prédateur.
        :param nouvelles_coords: Les nouvelles coordonnées du prédateur.
        '''
        _pred = self.d_preds.pop(vieilles_coords)
        _pred.x = nouvelles_coords[0]
        _pred.y = nouvelles_coords[1]
        
        self.d_preds[tuple(nouvelles_coords)] = _pred
        
    def chercher_positions_vides(self, key):
        '''
        Cherche des positions vides aléatoirement à 5 reprises avant
        d'abandonner.
        
        :param key: La clé de l'objet pour lequel chercher des déplacements.
        
        :retour: Les coordonnées actuelles et les nouvelles sous forme de tuple
                 comme (vieilles, nouvelles).
        '''
        _deplacements_possibles = self.d_preds[key].deplacements_possibles()
        coordonnees = ()
        
        for i in range(5):
            deplacement_aleatoire = _deplacements_possibles[np.random.randint(0, len(_deplacements_possibles))]

            if self.verifier_position(key, deplacement_aleatoire):
                nouvelles_coords = (
                    key[0] + deplacement_aleatoire[0],
                    key[1] + deplacement_aleatoire[1]
                )

                coordonnees = (key, nouvelles_coords)
                break 
                    
        return coordonnees
        
#(UNIQUES AUX PRÉDATEURS)#        
        
    def verifier_position_proies(self, coords, deplacement, _dict_proies):
        '''
        Vérifier si une position contient une proie.
        
        :param coords: coordonnées d'un prédateur (tuple)
        :param deplacement: deplacement à partir des coordonnées (tuple)
        :param _dict_proies: dictionnaire des proies (dict)
        '''
        position = (
            coords[0] + deplacement[0],
            coords[1] + deplacement[1]
            )
       
        return position in _dict_proies.d_proies
       
    def chercher_positions_proies(self, key, _dict_proies):
        '''
        Cherche une position contenant une proie aléatoirement à 5 reprises
        avant d'abandonner.
        
        :param key: la clé de l'objet pour lequel chercher des déplacements (tuple)
        :param _dict_proies: dictionnaire des proies (dict)
        
        :retour: Les coordonnées actuelles et les nouvelles sous forme de tuple
                 comme (vieilles, nouvelles).
        '''
        _deplacements_possibles = self.d_preds[key].deplacements_possibles()
        coordonnees = ()
    
        for i in range(5):
            deplacement_aleatoire = _deplacements_possibles[np.random.randint(0, len(_deplacements_possibles))]

            if self.verifier_position_proies(key, deplacement_aleatoire, _dict_proies):
                nouvelles_coords = (
                    key[0] + deplacement_aleatoire[0],
                    key[1] + deplacement_aleatoire[1]
                )

                coordonnees = (key, nouvelles_coords)
                break 
                    
        return coordonnees
    
    
###~~~FONCTIONS DE LA SIMULATION~~~###
       

    def eliminer_morts(self):
        '''
        Passer en revue les prédateurs et éliminer celles qui
        qualifient comme étant morts.
        '''
        a_enlever = []
        
        # trouver les prédateurs morts
        for key in self.d_preds.keys():
            if self.d_preds[key].est_mort():
                a_enlever.append(key)
                
        # éliminer les prédateurs morts
        for key in a_enlever:
            del self.d_preds[key]
    
    def diviser(self):
        '''
        Effectuer la division de chaque individu si son age,
        ses réserves et l'espace le permet.
        '''
        liste_keys = deepcopy(list(self.d_preds.keys()))
        
        for key in self.d_preds:
            if self.d_preds[key].age < pred.AGE_DIVISION or self.d_preds[key].reserves < pred.RESERVES_DIVISION:
                liste_keys.remove(key)
        
        for key in liste_keys:
            coords = self.chercher_positions_vides(key)
            
            if coords != ():
                reserves_moitie = self.d_preds[key].reserves / 2
            
                self.d_preds[coords[0]] = pred(coords[0][0], coords[0][1], 0, reserves_moitie)
                self.d_preds[coords[1]] = pred(coords[1][0], coords[1][1], 0, reserves_moitie)
                
           
    def incrementer_temps(self):
        '''
        Appliquer le vieillissement à chaque prédateur.
        '''
        for individu_pred in self.d_preds.values():
            individu_pred.vieillir()

#(UNIQUES AUX PRÉDATEURS)# 
            
    def manger(self, _dict_proies):
        '''
        Laisser les prédateurs manger une proie si ils 
        se trouvent à la même place.
        
        :param _dict_proies: dictionnaire des proies (dict)
        '''
        for key in self.d_preds:
            if key in _dict_proies.d_proies and self.d_preds[key].reserves < pred.RESERVES_SATIETE:
                del _dict_proies.d_proies[key]
                self.d_preds[key].reserves += 1
       
    def poursuivre_proies(self, _dict_proies):
        '''
        Faire déplacer les prédateurs à une case adjacente 
        contenant une proie si possible.
        
        :param _dict_proies: dictionnaire des proies (dict)
        '''
        liste_keys = deepcopy(list(self.d_preds.keys())) # pour modifier le dictionaire en TR
        
        for key in liste_keys:
            coordonnees = self.chercher_positions_proies(key, _dict_proies)
            
            if coordonnees != ():
                self.changer_position(coordonnees[0], coordonnees[1])

## Boucle principale

In [7]:
def run(n_iterations):
    '''
    Boucle principale de la simulation. Répète l'algorithme de simulation 
    sur le nombre d'itérations déterminé et tiens à jour une liste de 
    coordonnées de proies et de prédateurs.
    
    :param n_iterations: Le nombre d'itérations à effectuer dans la simulation.
    
    :retour: Retourne deux listes de coordonnées par itération, une pour chaque.
    '''
    _pos_proies = []
    _pos_preds = []

    proies = dict_proies(n_proie0)
    predateurs = dict_preds(n_pred0)
    
    # Pour capturer les conditions initiales
    _pos_proies.append(list(proies.d_proies.keys())) 
    _pos_preds.append(list(predateurs.d_preds.keys()))
    
    for i in range(n_iterations):

        proies.eliminer_morts()
        predateurs.eliminer_morts()


        proies.diviser()
        predateurs.diviser()

        
        predateurs.manger(proies)
        

        predateurs.poursuivre_proies(proies)
        
        
        proies.incrementer_temps()
        predateurs.incrementer_temps()
        
        _pos_proies.append(list(proies.d_proies.keys()))
        _pos_preds.append(list(predateurs.d_preds.keys()))
        
    return _pos_proies, _pos_preds

In [8]:
t_ini = timeit.default_timer()
pos_proies, pos_preds = run(n_iter)
temps_sim = timeit.default_timer() - t_ini

print(f'Durée de la simulation : {round(temps_sim)} secondes.')

Durée de la simulation : 209 secondes.


# Copie des données avec le module pickle

### Enregistrement des données calculées

In [8]:
nom_du_fichier = 'donnees_proies_preds'
data = [pos_proies, pos_preds]

# copier la liste dans un fichier binaire
with open(nom_du_fichier, 'wb') as f:
    pickle.dump(data, f)

NameError: name 'pos_proies' is not defined

### Extraction de données préalablement calculées

In [9]:
with open(nom_du_fichier, 'rb') as f:
    data_load = pickle.load(f)

In [10]:
# Vérification
print(len(data_load)) # deux listes (proies et prédateurs)
print(len(data_load[0])) # Une liste pour chaque itération
print(len(data_load[0][0])) # Une liste de tout les tuples de coordonnées
print(len(data_load[0][0][0])) # Les éléments du tuple

2
1001
193
2


In [11]:
# On sépare les données pour faciliter les manipulation
data_proies, data_preds = data_load

In [12]:
# On calcules les populations selon le temps
effectif_de_proies = []

for sous_liste in data_proies:
    effectif = len(sous_liste)
    effectif_de_proies.append(effectif)

effectif_de_preds = []

for sous_liste in data_preds:
    effectif = len(sous_liste)
    effectif_de_preds.append(effectif)

## Graphique de l'évolution des populations selon le temps

In [13]:
plt.figure()

# Grille de temps
t = np.arange(0, n_iter+1, 1)

plt.plot(t, effectif_de_proies)
plt.plot(t, effectif_de_preds)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Animation

In [2]:
%matplotlib ipympl
fig, ax = plt.subplots()

# Initialisation de l'affichage des proies à t = 0 sur le territoire
positions_proies_iter = data_proies[0] 
x, y = zip(*positions_proies_iter)
scat = ax.scatter(x,y, s=10)

# Initialisation de l'affichage des prédateurs à t = 0 sur le territoire
positions_pred_iter = data_preds[0] 
x, y = zip(*positions_pred_iter)
scat2 = ax.scatter(x,y, s=15 ,  color = "red", marker='s')

# Fonction exécutée lors de l'animation
def anim(t):

    # Rafraichir la position des proies
    positions_proies_iter = data_proies[t]
    x, y = zip(*positions_proies_iter)
    dim = np.array([x,y]).T
    scat.set_offsets(dim)
    
    # Rafraichir la position des prédateurs
    positions_pred_iter = data_preds[t] 
    x, y = zip(*positions_pred_iter)
    dim = np.array([x,y]).T
    scat2.set_offsets(dim)
    
    # On affiche le temps dans le titre
    plt.title(f"{t}")

# AFficher l'annimation
ani = animation.FuncAnimation(fig, anim, frames=np.arange(0, 1001, 1), interval = 100, repeat=False)

NameError: name 'plt' is not defined

In [16]:
# Enregistrer l'animation en format .mp4
writer = FFMpegWriter(fps=30)

ani.save('animation_proies_preds.mp4', writer)